In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

# Check if TensorFlow can detect a GPU
gpu_device_name = tf.test.gpu_device_name()

if gpu_device_name:
    print('GPU device found:', gpu_device_name)
else:
    print("No GPU available. Using CPU instead.")

No GPU available. Using CPU instead.


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [4]:
import warnings
warnings.filterwarnings("ignore")

# Data Loading (with noise ) and preprocessing

In [ ]:
train_data= pd.read_csv("/content/drive/MyDrive/AML/A2/cf_train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/AML/A2/cf_test.csv")

In [ ]:
train_data.shape, test_data.shape

((62400, 31), (62400, 31))

In [ ]:
# Define the mapping of values to indices
value_to_index = {0.00: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.00: 4}

# Replace values with corresponding indices
train_data["target_10_val"] = train_data["target_10_val"].replace(value_to_index).astype(int)
test_data["target_10_val"] = test_data["target_10_val"].replace(value_to_index).astype(int)

In [ ]:
label_encoder = LabelEncoder()

train_data["sigma"] = label_encoder.fit_transform(train_data["sigma"])
test_data["sigma"] = label_encoder.fit_transform(test_data["sigma"])

In [ ]:
# Initialize scaler for feature scaling
scaler = StandardScaler()

# Train the model on cf_train.csv
X_train = train_data.drop(columns=["era", "day", "target_10_val"])
y_train = train_data["target_10_val"]
X_train_scaled = scaler.fit_transform(X_train)

# MAML (Model-Agnostic Meta Learning)

In [5]:
# Define model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [6]:
# Define MAML hyperparameters
learning_rate_inner = 0.01
learning_rate_outer = 0.001
num_inner_updates = 5
batch_size_outer = 32

In [7]:
# Define loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate_outer)

In [ ]:
# Ensure X_train has the correct shape
X_train_arr = np.array(X_train)  # Convert to numpy array if not already
X_train_arr = X_train_arr.reshape(-1, 28)  # R

In [ ]:
# Pre-train MAML
for _ in range(100):  # Number of pre-training epochs
    with tf.GradientTape() as tape:
        y_pred = model(X_train_arr)
        loss = loss_fn(y_train, y_pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
# Continual learning loop
predictions = []
for i, test_row in test_data[:10000].iterrows():
    if i < 15:
      predictions.append(3)
      continue

    # Adapt model to new task on using 5 rows from i-10 to i - 5
    X_task = scaler.transform(test_data.iloc[i-10:i-5].drop(columns=["era", "day", "target_10_val"]))
    y_task = test_data.iloc[i-10:i-5]["target_10_val"]
    inner_optimizer = tf.keras.optimizers.Adam(learning_rate_inner)

    # Ensure X_train has the correct shape
    X_task = np.array(X_task)  # Convert to numpy array if not already
    X_task = X_task.reshape(-1, 28)  # R

    for _ in range(num_inner_updates):
        with tf.GradientTape() as tape:
            y_pred_task = model(X_task)
            task_loss = loss_fn(y_task, y_pred_task)
        gradients_task = tape.gradient(task_loss, model.trainable_variables)
        inner_optimizer.apply_gradients(zip(gradients_task, model.trainable_variables))

    # Predict on the current sample
    X_current = scaler.transform(test_row.drop(columns=["era", "day", "target_10_val"])[X_train.columns].values.reshape(1, -1))
    y_pred_current = model(X_current)
    y_pred = np.argmax(y_pred_current, axis=1)[0]  # Extract the predicted class
    predictions.append(y_pred)


In [ ]:
# # Evaluate accuracy
true_labels = test_data["target_10_val"]
accuracy = np.mean(predictions == true_labels[:len(predictions)])
print("Accuracy:", accuracy)

Accuracy: 0.4833


# Without Noise

### Dataset loading and prepreocessing

In [8]:
train_data= pd.read_csv("/content/drive/MyDrive/AML/A2/cf_train_no_noise.csv")
test_data = pd.read_csv("/content/drive/MyDrive/AML/A2/cf_test_no_noise.csv")

In [9]:
# Define the mapping of values to indices
value_to_index = {0.00: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.00: 4}

# Replace values with corresponding indices
train_data["target_10_val"] = train_data["target_10_val"].replace(value_to_index).astype(int)
test_data["target_10_val"] = test_data["target_10_val"].replace(value_to_index).astype(int)

label_encoder = LabelEncoder()

train_data["sigma"] = label_encoder.fit_transform(train_data["sigma"])
test_data["sigma"] = label_encoder.fit_transform(test_data["sigma"])

# Initialize scaler for feature scaling
scaler = StandardScaler()

# Train the model on cf_train.csv
X_train = train_data.drop(columns=["era", "day", "target_10_val"])
y_train = train_data["target_10_val"]
X_train_scaled = scaler.fit_transform(X_train)


In [12]:
# Ensure X_train has the correct shape
X_train_arr = np.array(X_train_scaled)  # Convert to numpy array if not already
X_train_arr = X_train_arr.reshape(-1, 28)  # R


# Pre-train MAML
for _ in range(100):  # Number of pre-training epochs
    with tf.GradientTape() as tape:
        y_pred = model(X_train_arr)
        loss = loss_fn(y_train, y_pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))


In [13]:
# Continual learning loop
predictions = []
for i, test_row in test_data[:10000].iterrows():
    if i < 15:
      predictions.append(3)
      continue

    # Adapt model to new task on using 5 rows from i-10 to i - 5
    X_task = scaler.transform(test_data.iloc[i-10:i-5].drop(columns=["era", "day", "target_10_val"]))
    y_task = test_data.iloc[i-10:i-5]["target_10_val"]
    inner_optimizer = tf.keras.optimizers.Adam(learning_rate_inner)

    # Ensure X_train has the correct shape
    X_task = np.array(X_task)  # Convert to numpy array if not already
    X_task = X_task.reshape(-1, 28)  # R

    for _ in range(num_inner_updates):
        with tf.GradientTape() as tape:
            y_pred_task = model(X_task)
            task_loss = loss_fn(y_task, y_pred_task)
        gradients_task = tape.gradient(task_loss, model.trainable_variables)
        inner_optimizer.apply_gradients(zip(gradients_task, model.trainable_variables))

    # Predict on the current sample
    X_current = scaler.transform(test_row.drop(columns=["era", "day", "target_10_val"])[X_train.columns].values.reshape(1, -1))
    y_pred_current = model(X_current)
    y_pred = np.argmax(y_pred_current, axis=1)[0]  # Extract the predicted class
    predictions.append(y_pred)


In [15]:
# # Evaluate accuracy
true_labels = test_data["target_10_val"]
accuracy = np.mean(predictions == true_labels[:len(predictions)])
print("Accuracy:", accuracy)

Accuracy: 0.8576
